# TAREA 5

In [2]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://archive.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Ign:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,071 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,425 kB]

In [3]:
pip install findspark

In [1]:
import matplotlib.pyplot as plt
import statsmodels.api as sm
import numbers
import numpy as np
import pandas as pd
from tabulate import tabulate
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score
from typing import List, Union, Dict, Any, Tuple
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge
from matplotlib.figure import Figure
from matplotlib.axes import Axes

In [2]:
import os
import pandas as pd
import setuptools
import sys
import findspark
findspark.init()
findspark.find()
from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F
import time
from pyspark.sql.functions import col, to_date, concat, lpad, lit

In [3]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import ChiSqSelector

In [4]:
from pyspark import SparkContext
from pyspark.ml.feature import StandardScaler

In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
               .appName('ml') \
               .getOrCreate()

In [103]:
df = spark.read.csv('public_emdat_request.csv', header=True, inferSchema=True)

In [30]:
def clean_column_names(df):
    new_columns = [col.replace(" ", "_").lower().strip() for col in df.columns]
    df = df.toDF(*new_columns)
    return df

In [104]:
df = clean_column_names(df)

In [105]:
df= df.drop("disno", "external_ids", "origin","associated_types","ofda_bha_response",
            "appeal","declaration","aid_contribution","latitude","longitude","river_basin",
            "injured","affected","homeless","cpi","classification","entry_date","last_update","classification_key",
            "magnitude_scale","event_name")
df.printSchema()

root
 |-- historic: string (nullable = true)
 |-- disaster_group: string (nullable = true)
 |-- disaster_subgroup: string (nullable = true)
 |-- disaster_type: string (nullable = true)
 |-- disaster_subtype: string (nullable = true)
 |-- iso: string (nullable = true)
 |-- country: string (nullable = true)
 |-- subregion: string (nullable = true)
 |-- region: string (nullable = true)
 |-- location: string (nullable = true)
 |-- magnitude: double (nullable = true)
 |-- start_year: integer (nullable = true)
 |-- start_month: integer (nullable = true)
 |-- start_day: integer (nullable = true)
 |-- end_year: integer (nullable = true)
 |-- end_month: integer (nullable = true)
 |-- end_day: integer (nullable = true)
 |-- total_deaths: integer (nullable = true)
 |-- total_affected: integer (nullable = true)
 |-- insured_damage_adjusted: integer (nullable = true)



In [106]:
df.show()

+--------+--------------+-----------------+-----------------+--------------------+---+--------------------+--------------------+--------+--------------------+---------+----------+-----------+---------+--------+---------+-------+------------+--------------+-----------------------+
|historic|disaster_group|disaster_subgroup|    disaster_type|    disaster_subtype|iso|             country|           subregion|  region|            location|magnitude|start_year|start_month|start_day|end_year|end_month|end_day|total_deaths|total_affected|insured_damage_adjusted|
+--------+--------------+-----------------+-----------------+--------------------+---+--------------------+--------------------+--------+--------------------+---------+----------+-----------+---------+--------+---------+-------+------------+--------------+-----------------------+
|     Yes|       Natural|   Meteorological|            Storm|    Tropical cyclone|USA|United States of ...|    Northern America|Americas|   Galveston (Texas)

In [107]:
def create_date_column(df, year_col, month_col, day_col, new_col_name):
    df = df.withColumn(month_col, lpad(col(month_col), 2, "0")) \
           .withColumn(day_col, lpad(col(day_col), 2, "0"))

    return df.withColumn(new_col_name, to_date(concat(col(year_col),
                                                        lit("-"),
                                                        col(month_col),
                                                        lit("-"),
                                                        col(day_col)),
                                                "yyyy-MM-dd"))

df = create_date_column(df, "start_year", "start_month", "start_day", "start_date")
df = create_date_column(df, "end_year", "end_month", "end_day", "end_date")

In [108]:
#Dropna
columns_to_dropna = ["location", "start_date","magnitude"]
df = df.dropna(subset=columns_to_dropna)

#Fillna
fill_values = {
    #"event_name": "None",
    #"magnitude": 0,
    "total_deaths":0,
    "total_affected":0,
    "insured_damage_adjusted":0
}
for column, value in fill_values.items():
    df = df.fillna({column: value})

#df_indexado = df.filter(~((col("disaster_type").isin("Drought", "Earthquake", "Epidemic",
#                                                                 "Extreme temperature", "Flood",
#                                                                 "Storm", "Wildfire")) & col("magnitude").isNull()))

# Mostrar el esquema final del DataFrame
df = df.withColumn("end_month", col("end_month").cast("integer"))
df = df.withColumn("start_month", col("start_month").cast("integer"))
df = df.withColumn("end_day", col("end_day").cast("integer"))
df = df.withColumn("start_day", col("start_day").cast("integer"))
df.printSchema()


root
 |-- historic: string (nullable = true)
 |-- disaster_group: string (nullable = true)
 |-- disaster_subgroup: string (nullable = true)
 |-- disaster_type: string (nullable = true)
 |-- disaster_subtype: string (nullable = true)
 |-- iso: string (nullable = true)
 |-- country: string (nullable = true)
 |-- subregion: string (nullable = true)
 |-- region: string (nullable = true)
 |-- location: string (nullable = true)
 |-- magnitude: double (nullable = true)
 |-- start_year: integer (nullable = true)
 |-- start_month: integer (nullable = true)
 |-- start_day: integer (nullable = true)
 |-- end_year: integer (nullable = true)
 |-- end_month: integer (nullable = true)
 |-- end_day: integer (nullable = true)
 |-- total_deaths: integer (nullable = false)
 |-- total_affected: integer (nullable = false)
 |-- insured_damage_adjusted: integer (nullable = false)
 |-- start_date: date (nullable = true)
 |-- end_date: date (nullable = true)



In [109]:
column_names = df.columns
print(column_names)

['historic', 'disaster_group', 'disaster_subgroup', 'disaster_type', 'disaster_subtype', 'iso', 'country', 'subregion', 'region', 'location', 'magnitude', 'start_year', 'start_month', 'start_day', 'end_year', 'end_month', 'end_day', 'total_deaths', 'total_affected', 'insured_damage_adjusted', 'start_date', 'end_date']


In [110]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType
import pandas as pd

columns_to_index = ["historic", "disaster_group", "disaster_subgroup", "disaster_type", "disaster_subtype",
                    "iso", "country", "subregion", "region", "location"]

indexers = [StringIndexer(inputCol=col, outputCol=f"{col}_indexado", handleInvalid='skip') for col in columns_to_index]
pipeline = Pipeline(stages=indexers)
model = pipeline.fit(df)
df_indexado = model.transform(df)

# Ajustar índices y eliminar columnas originales
for col_name in columns_to_index:
    index_col = f"{col_name}_indexado"
    if index_col in df_indexado.columns:
        # Crear nueva columna que ajusta el índice
        df_indexado = df_indexado.withColumn(index_col, col(index_col) + 1)
        # Eliminar la columna original si es necesario
        df_indexado = df_indexado.drop(col_name)

# Convertir a enteros
for col_name in df_indexado.columns:
    if "_indexado" in col_name:
        df_indexado = df_indexado.withColumn(col_name, col(col_name).cast(IntegerType()))

df_indexado.printSchema()

glosario = []
for indexer in indexers:
    input_col = indexer.getInputCol()
    output_col = indexer.getOutputCol()
    index_labels = model.stages[columns_to_index.index(input_col)].labels

    for idx, label in enumerate(index_labels):
        glosario.append((output_col, idx + 1, label))  # Sumar 1 al índice

glos = pd.DataFrame(glosario, columns=['Columna', 'Índice', 'Definición'])
glos.to_excel('glosario.xlsx', index=False)



root
 |-- magnitude: double (nullable = true)
 |-- start_year: integer (nullable = true)
 |-- start_month: integer (nullable = true)
 |-- start_day: integer (nullable = true)
 |-- end_year: integer (nullable = true)
 |-- end_month: integer (nullable = true)
 |-- end_day: integer (nullable = true)
 |-- total_deaths: integer (nullable = false)
 |-- total_affected: integer (nullable = false)
 |-- insured_damage_adjusted: integer (nullable = false)
 |-- start_date: date (nullable = true)
 |-- end_date: date (nullable = true)
 |-- historic_indexado: integer (nullable = true)
 |-- disaster_group_indexado: integer (nullable = true)
 |-- disaster_subgroup_indexado: integer (nullable = true)
 |-- disaster_type_indexado: integer (nullable = true)
 |-- disaster_subtype_indexado: integer (nullable = true)
 |-- iso_indexado: integer (nullable = true)
 |-- country_indexado: integer (nullable = true)
 |-- subregion_indexado: integer (nullable = true)
 |-- region_indexado: integer (nullable = true)
 |

In [111]:
columns_to_drop = ["start_date", "end_date"]
df_indexado = df_indexado.drop(*columns_to_drop)

In [112]:
df_indexado.printSchema()

root
 |-- magnitude: double (nullable = true)
 |-- start_year: integer (nullable = true)
 |-- start_month: integer (nullable = true)
 |-- start_day: integer (nullable = true)
 |-- end_year: integer (nullable = true)
 |-- end_month: integer (nullable = true)
 |-- end_day: integer (nullable = true)
 |-- total_deaths: integer (nullable = false)
 |-- total_affected: integer (nullable = false)
 |-- insured_damage_adjusted: integer (nullable = false)
 |-- historic_indexado: integer (nullable = true)
 |-- disaster_group_indexado: integer (nullable = true)
 |-- disaster_subgroup_indexado: integer (nullable = true)
 |-- disaster_type_indexado: integer (nullable = true)
 |-- disaster_subtype_indexado: integer (nullable = true)
 |-- iso_indexado: integer (nullable = true)
 |-- country_indexado: integer (nullable = true)
 |-- subregion_indexado: integer (nullable = true)
 |-- region_indexado: integer (nullable = true)
 |-- location_indexado: integer (nullable = true)



In [17]:
from pyspark.ml.feature import VectorAssembler, ChiSqSelector

In [113]:
df_indexado = df_indexado.dropna()

In [114]:
df_indexado_pd = df_indexado.toPandas()
df_indexado_pd.to_excel('df_indexado_2.xlsx', index=False)

In [148]:
df_indexado_2 = df_indexado.limit(100)
df_indexado_2.show()

+---------+----------+-----------+---------+--------+---------+-------+------------+--------------+-----------------------+-----------------+-----------------------+--------------------------+----------------------+-------------------------+------------+----------------+------------------+---------------+-----------------+
|magnitude|start_year|start_month|start_day|end_year|end_month|end_day|total_deaths|total_affected|insured_damage_adjusted|historic_indexado|disaster_group_indexado|disaster_subgroup_indexado|disaster_type_indexado|disaster_subtype_indexado|iso_indexado|country_indexado|subregion_indexado|region_indexado|location_indexado|
+---------+----------+-----------+---------+--------+---------+-------+------------+--------------+-----------------------+-----------------+-----------------------+--------------------------+----------------------+-------------------------+------------+----------------+------------------+---------------+-----------------+
|    220.0|      1900|   

In [149]:
inputCols = [
    #'event_name',
    'magnitude',
    #'magnitude_scale',
    #'start_year',
    #'start_month',
    #'start_day',
    #'start_date',
    #'end_year',
    #'end_month',
    #'end_day',
    #'end_date',
    #'total_deaths',
    #'total_affected',
    #'insured_damage_adjusted',
    #'historic_indexado',
    'disaster_group_indexado',
    ##'disaster_subgroup_indexado',
    'disaster_type_indexado',
    'disaster_subtype_indexado',
    #'iso_indexado',
    ##'country_indexado',
    #subregion_indexado',
    'region_indexado',
    #'location_indexado'
]
assembler = VectorAssembler(inputCols = inputCols, outputCol = "features")
df_with_features = assembler.transform(df_indexado_2)

df_with_features.select('features').show(10, False)

+-----------------------+
|features               |
+-----------------------+
|[220.0,1.0,3.0,3.0,2.0]|
|[5.9,1.0,2.0,1.0,1.0]  |
|[7.9,1.0,2.0,12.0,1.0] |
|[6.4,1.0,2.0,1.0,3.0]  |
|[7.5,1.0,2.0,1.0,2.0]  |
|[6.9,1.0,2.0,1.0,3.0]  |
|[7.7,1.0,2.0,1.0,1.0]  |
|[6.3,1.0,2.0,1.0,1.0]  |
|[5.7,1.0,2.0,1.0,1.0]  |
|[5.8,1.0,2.0,1.0,1.0]  |
+-----------------------+
only showing top 10 rows



In [150]:
df_with_features.printSchema()

root
 |-- magnitude: double (nullable = true)
 |-- start_year: integer (nullable = true)
 |-- start_month: integer (nullable = true)
 |-- start_day: integer (nullable = true)
 |-- end_year: integer (nullable = true)
 |-- end_month: integer (nullable = true)
 |-- end_day: integer (nullable = true)
 |-- total_deaths: integer (nullable = false)
 |-- total_affected: integer (nullable = false)
 |-- insured_damage_adjusted: integer (nullable = false)
 |-- historic_indexado: integer (nullable = true)
 |-- disaster_group_indexado: integer (nullable = true)
 |-- disaster_subgroup_indexado: integer (nullable = true)
 |-- disaster_type_indexado: integer (nullable = true)
 |-- disaster_subtype_indexado: integer (nullable = true)
 |-- iso_indexado: integer (nullable = true)
 |-- country_indexado: integer (nullable = true)
 |-- subregion_indexado: integer (nullable = true)
 |-- region_indexado: integer (nullable = true)
 |-- location_indexado: integer (nullable = true)
 |-- features: vector (nullabl

In [151]:
df_with_features.show(7)

+---------+----------+-----------+---------+--------+---------+-------+------------+--------------+-----------------------+-----------------+-----------------------+--------------------------+----------------------+-------------------------+------------+----------------+------------------+---------------+-----------------+--------------------+
|magnitude|start_year|start_month|start_day|end_year|end_month|end_day|total_deaths|total_affected|insured_damage_adjusted|historic_indexado|disaster_group_indexado|disaster_subgroup_indexado|disaster_type_indexado|disaster_subtype_indexado|iso_indexado|country_indexado|subregion_indexado|region_indexado|location_indexado|            features|
+---------+----------+-----------+---------+--------+---------+-------+------------+--------------+-----------------------+-----------------+-----------------------+--------------------------+----------------------+-------------------------+------------+----------------+------------------+---------------+--

In [152]:
selector = ChiSqSelector(numTopFeatures = 3, featuresCol = "features", labelCol="total_affected",
                         outputCol="selected_features")
df_sel = selector.fit(df_with_features).transform(df_with_features)
df_sel.show(5)

+---------+----------+-----------+---------+--------+---------+-------+------------+--------------+-----------------------+-----------------+-----------------------+--------------------------+----------------------+-------------------------+------------+----------------+------------------+---------------+-----------------+--------------------+-----------------+
|magnitude|start_year|start_month|start_day|end_year|end_month|end_day|total_deaths|total_affected|insured_damage_adjusted|historic_indexado|disaster_group_indexado|disaster_subgroup_indexado|disaster_type_indexado|disaster_subtype_indexado|iso_indexado|country_indexado|subregion_indexado|region_indexado|location_indexado|            features|selected_features|
+---------+----------+-----------+---------+--------+---------+-------+------------+--------------+-----------------------+-----------------+-----------------------+--------------------------+----------------------+-------------------------+------------+----------------+-

In [22]:
from pyspark.ml.feature import StandardScaler

In [153]:
scaler = StandardScaler(inputCol="features", outputCol="features_scaled")
df_escalado = scaler.fit(df_with_features).transform(df_with_features)

In [144]:
df_escalado.show(7)

+---------+----------+-----------+---------+--------+---------+-------+------------+--------------+-----------------------+-----------------+-----------------------+--------------------------+----------------------+-------------------------+------------+----------------+------------------+---------------+-----------------+--------------------+--------------------+
|magnitude|start_year|start_month|start_day|end_year|end_month|end_day|total_deaths|total_affected|insured_damage_adjusted|historic_indexado|disaster_group_indexado|disaster_subgroup_indexado|disaster_type_indexado|disaster_subtype_indexado|iso_indexado|country_indexado|subregion_indexado|region_indexado|location_indexado|            features|     features_scaled|
+---------+----------+-----------+---------+--------+---------+-------+------------+--------------+-----------------------+-----------------+-----------------------+--------------------------+----------------------+-------------------------+------------+------------

In [24]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [154]:
# Dividir datos en entrenamiento y prueba
train, test = df_escalado.randomSplit([0.7, 0.3])

In [155]:
train.show()

+---------+----------+-----------+---------+--------+---------+-------+------------+--------------+-----------------------+-----------------+-----------------------+--------------------------+----------------------+-------------------------+------------+----------------+------------------+---------------+-----------------+--------------------+--------------------+
|magnitude|start_year|start_month|start_day|end_year|end_month|end_day|total_deaths|total_affected|insured_damage_adjusted|historic_indexado|disaster_group_indexado|disaster_subgroup_indexado|disaster_type_indexado|disaster_subtype_indexado|iso_indexado|country_indexado|subregion_indexado|region_indexado|location_indexado|            features|     features_scaled|
+---------+----------+-----------+---------+--------+---------+-------+------------+--------------+-----------------------+-----------------+-----------------------+--------------------------+----------------------+-------------------------+------------+------------

In [156]:
#from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(featuresCol="features_scaled", labelCol="total_affected")

model = lr.fit(train)


In [157]:
# Evaluar el modelo
predictions = model.transform(test)
evaluator = MulticlassClassificationEvaluator(labelCol="total_affected", predictionCol="prediction",
                                              metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
accuracy

0.5625

In [160]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler

# Load and prepare the dataset
data = df_indexado#, header=True, inferSchema=True
feature_columns = ['magnitude', 'disaster_group_indexado', 'region_indexado']  # assuming these are the features
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
data = assembler.transform(data)

# Split the data into training and test sets
train_data, test_data = data.randomSplit([0.7, 0.3])

# Train the model
lr = LinearRegression(featuresCol="features", labelCol="total_affected")
model = lr.fit(train_data)

# Make predictions
predictions = model.transform(test_data)
predictions.select("prediction", "total_affected").show()

+------------------+--------------+
|        prediction|total_affected|
+------------------+--------------+
|122259.42973250139|            11|
| 451270.1557320559|             0|
|122286.26966148498|           263|
|122286.26966148498|         31500|
|122287.94715704652|             0|
|122292.97964373091|             0|
| 451285.2531921092|       1839888|
| 122303.0446170998|             0|
| 122303.0446170998|          2220|
| 451291.9631743551|             0|
|122304.72211266134|             0|
|122304.72211266134|             0|
|122304.72211266134|             0|
| 780282.5592271718|             0|
|451295.31816547806|         25475|
|  451298.673156601|             0|
|122311.43209490727|             0|
|122311.43209490727|             0|
|122314.78708603024|             0|
|122326.52955496055|             0|
+------------------+--------------+
only showing top 20 rows



In [163]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import StringIndexer, VectorAssembler

# Load the Iris dataset
data = df_indexado_2

# Assemble features
feature_columns = ['magnitude', 'disaster_group_indexado']
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
data = assembler.transform(data)

# Split the data
train_data, test_data = data.randomSplit([0.7, 0.3])

# Train the model
rf = RandomForestClassifier(featuresCol="features", labelCol="total_affected")
model = rf.fit(train_data)

# Predictions
predictions = model.transform(test_data)
predictions.select("prediction", "total_affected").show()

Py4JJavaError: An error occurred while calling o7397.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 2251.0 failed 1 times, most recent failure: Lost task 0.0 in stage 2251.0 (TID 1321) (DESKTOP-QOP6DNJ executor driver): java.lang.RuntimeException: Labels MUST be in [0, 100), but got 669.0
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage2.hashAgg_doAggregate_max_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage2.hashAgg_doConsume_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage2.hashAgg_doAggregateWithoutKey_1$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage2.hashAgg_doAggregateWithoutKey_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage2.processNe